## Crypto Sentiment Analyzer

In [45]:
# Imports
import os
import requests
import json
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
%matplotlib inline

In [46]:
load_dotenv()

True

## Establish Baseline Performance

#### Step 1: Import the OHLCV dataset into a Pandas DataFrame.

In [47]:
# Import the OHLCV dataset into a 

btc_url = 'https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD'
eth_url = 'https://api.alternative.me/v2/ticker/Ethereum/?convert=USD'
bnb_url = 'https://api.alternative.me/v2/ticker/BinanceCoin/?convert=USD'
xrp_url = 'https://api.alternative.me/v2/ticker/ripple/?convert=USD'
luna_url = 'https://api.alternative.me/v2/ticker/terra-luna/?convert=USD'
cardano_url ='https://api.alternative.me/v2/ticker/cardano/?convert=USD'
avax_url = 'https://api.alternative.me/v2/ticker/avalanche-2/?convert=USD'
dot_url = 'https://api.alternative.me/v2/ticker/polkadot/?convert=USD'
doge_url = 'https://api.alternative.me/v2/ticker/doge/?convert=USD'
shiba_url= 'https://api.alternative.me/v2/ticker/shiba-inu/?convert=USD'
api_key = os.getenv('API_Key')

In [48]:
btc_response = requests.get(btc_url).json()
eth_response= requests.get(eth_url).json()
bnb_response= requests.get(bnb_url).json()
xrp_response= requests.get(xrp_url).json()
luna_response = requests.get(luna_url).json()
cardano_response = requests.get(cardano_url).json()
avax_response = requests.get(avax_url).json()
dot_response= requests.get(dot_url).json()
doge_response = requests.get(doge_url).json()
shiba_response = requests.get(shiba_url).json()



In [72]:
alpaca_api_key= os.getenv('API_Key')
alpaca_secret_key = os.getenv('API_Secret_Key')


alpaca= tradeapi.REST(
alpaca_api_key,
alpaca_secret_key)

In [73]:

# Filter the date index and close columns
tickers = ['BTCUSD']
# Use the pct_change function to generate returns from close prices
timeframe = '1Day'

limit_rows= 1000
# Drop all NaN values from the DataFrame

start_date = pd.Timestamp('2018-01-01', tz= "America/New_York").isoformat()
end_date = pd.Timestamp('2022-03-15', tz="America/New_York").isoformat()


In [74]:
start_date

'2018-01-01T00:00:00-05:00'

In [77]:
from alpaca_trade_api.rest import TimeFrame, TimeFrameUnit
prices_df = alpaca.get_crypto_bars(
tickers,
timeframe, 
start=start_date,
end=end_date,
exchanges="CBSE").df

prices_df.head(100)

,exchange,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,,
2018-01-01 06:00:00+00:00,CBSE,13480.00,13999.00,13005.00,13194.02,6200.047084,49282,13515.969795,BTCUSD
2018-01-02 06:00:00+00:00,CBSE,14781.52,15198.88,14761.00,15098.88,4422.372928,36177,15002.191483,BTCUSD
2018-01-03 06:00:00+00:00,CBSE,15098.87,15400.00,14628.00,15098.14,12001.450099,91543,14992.624035,BTCUSD
2018-01-04 06:00:00+00:00,CBSE,15145.00,15176.86,14819.78,15038.00,3086.394104,26818,15022.817179,BTCUSD
2018-01-05 06:00:00+00:00,CBSE,16960.01,16999.00,16410.75,16594.08,4507.400878,43035,16696.958061,BTCUSD
...,...,...,...,...,...,...,...,...,...
2018-04-06 05:00:00+00:00,CBSE,6619.01,6900.00,6605.08,6842.40,2763.138179,11135,6799.623337,BTCUSD
2018-04-07 05:00:00+00:00,CBSE,6894.01,6990.00,6894.00,6969.00,1139.746521,6145,6950.619440,BTCUSD
2018-04-08 05:00:00+00:00,CBSE,7020.01,7163.03,7020.00,7135.13,2735.154272,11367,7105.749330,BTCUSD


In [78]:
prices_df.shape

(1534, 9)

In [79]:
project_df = prices_df.loc[:,["close"]]

In [80]:
project_df

,close
timestamp,
2018-01-01 06:00:00+00:00,13194.02
2018-01-02 06:00:00+00:00,15098.88
2018-01-03 06:00:00+00:00,15098.14
2018-01-04 06:00:00+00:00,15038.00
2018-01-05 06:00:00+00:00,16594.08
...,...
2022-03-10 06:00:00+00:00,38783.79
2022-03-11 06:00:00+00:00,39178.11
2022-03-12 06:00:00+00:00,39128.83


 #### Step 3: Split the data into training and testing datasets.

In [81]:
# Calculate the daily returns using the closing prices and the pct_change function
project_df["DailyReturn"] = project_df["close"].pct_change()

# Display sample data
project_df.head()

,close,DailyReturn
timestamp,,
2018-01-01 06:00:00+00:00,13194.02,NaN
2018-01-02 06:00:00+00:00,15098.88,0.144373
2018-01-03 06:00:00+00:00,15098.14,-0.000049
2018-01-04 06:00:00+00:00,15038.00,-0.003983
2018-01-05 06:00:00+00:00,16594.08,0.103477


In [82]:
# Drop all NaN values from the DataFrame
project_df = project_df.dropna()

# Review the DataFrame
display(project_df.head())
display(project_df.tail())

,close,DailyReturn
timestamp,,
2018-01-02 06:00:00+00:00,15098.88,0.144373
2018-01-03 06:00:00+00:00,15098.14,-0.000049
2018-01-04 06:00:00+00:00,15038.00,-0.003983
2018-01-05 06:00:00+00:00,16594.08,0.103477
2018-01-06 06:00:00+00:00,17098.99,0.030427


,close,DailyReturn
timestamp,,
2022-03-10 06:00:00+00:00,38783.79,-0.011152
2022-03-11 06:00:00+00:00,39178.11,0.010167
2022-03-12 06:00:00+00:00,39128.83,-0.001258
2022-03-13 06:00:00+00:00,38535.33,-0.015168
2022-03-14 05:00:00+00:00,38794.04,0.006714


#### Step 5: Review the classification report associated with the SVC model predictions.

In [83]:
# Initialize investment signal tally

investment_signal = 0

# Initialize trade_type column for buys and sell
project_df["score"] = np.nan

# Initialize variable to hold the previous_price
previous_price = 0

# Loop through the Pandas DataFrame and initiate a score for each iteration


for index, row in project_df.iterrows():
    
    # add 1 if the current day price is greater than the previous day
    
    if row["close"] < previous_price:
        investment_signal = investment_signal + 2
        project_df.loc[index, "score"] = 1
        
    if row["close"] < previous_price:
        investment_signal = investment_signal + 1
        project_df.loc[index, "score"] = 1    
          
    # subtract 1 if the current day price is greater than the previous day
    if row["close"] > previous_price:
        investment_signal = investment_signal - 1
        project_df.loc[index, "score"] = 0    
    
    # set the previous_price variable to the close of current row
    previous_price = row["close"]

    # if the index is the last index of the Data
    if index == project_df.index[-1]:
        project_df.loc[index, "score"] = 1

/Users/pattenwilliams/opt/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/pattenwilliams/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/pattenwilliams/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documenta

In [84]:
project_df.fillna
project_df.head(50)


,close,DailyReturn,score
timestamp,,,
2018-01-02 06:00:00+00:00,15098.88,0.144373,0.0
2018-01-03 06:00:00+00:00,15098.14,-0.000049,1.0
2018-01-04 06:00:00+00:00,15038.00,-0.003983,1.0
2018-01-05 06:00:00+00:00,16594.08,0.103477,0.0
2018-01-06 06:00:00+00:00,17098.99,0.030427,0.0
2018-01-07 06:00:00+00:00,15862.90,-0.072290,1.0
2018-01-08 06:00:00+00:00,14993.74,-0.054792,1.0
2018-01-09 06:00:00+00:00,14227.00,-0.051137,1.0
2018-01-10 06:00:00+00:00,13193.72,-0.072628,1.0


In [85]:
investment_signal

1375


#### Tune the Baseline Trading Algorithm


In [86]:
# Initialize trade_type column for buys and sell
project_df["trade_type"]= np.nan


# Loop through the Pandas DataFrame and initiate
for index, row in project_df.iterrows():
    
    # buy if the previous price is 0
    if previous_price == 0:
        project_df.loc[index, "trade_type"] = 1

    # buy if the current day price is less than previous close
    elif row["close"] < previous_price:
        project_df.loc[index, "trade_type"] = 1

    # hold if the current day price is greater than previous close
    elif row["close"] > previous_price:
        project_df.loc[index, "trade_type"] = 0

    # as well, hold if the current day price is equal to previous close
    else:
        project_df.loc[index, "trade_type"] = 0
    
    # set the previous_price variable to the close of current row
    previous_price = row["close"]

    # if the index is the last index of the Data
    if index == project_df.index[-1]:
        project_df.loc[index, "trade_type"] = 1

/Users/pattenwilliams/opt/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/pattenwilliams/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/pattenwilliams/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documenta

In [87]:
project_df

,close,DailyReturn,score,trade_type
timestamp,,,,
2018-01-02 06:00:00+00:00,15098.88,0.144373,0.0,1.0
2018-01-03 06:00:00+00:00,15098.14,-0.000049,1.0,1.0
2018-01-04 06:00:00+00:00,15038.00,-0.003983,1.0,1.0
2018-01-05 06:00:00+00:00,16594.08,0.103477,0.0,0.0
2018-01-06 06:00:00+00:00,17098.99,0.030427,0.0,0.0
...,...,...,...,...
2022-03-10 06:00:00+00:00,38783.79,-0.011152,1.0,1.0
2022-03-11 06:00:00+00:00,39178.11,0.010167,0.0,0.0
2022-03-12 06:00:00+00:00,39128.83,-0.001258,1.0,1.0


In [88]:
# Preview the DataFrame
project_df.head()

# Count how many firms are in each category
project_df['close'].value_counts()

3800.01     2
3690.00     2
6404.87     2
6243.54     1
8131.02     1
           ..
9830.00     1
37340.77    1
5226.87     1
11555.83    1
3859.99     1
Name: close, Length: 1530, dtype: int64

In [89]:
project_df.dropna

<bound method DataFrame.dropna of                               close  DailyReturn  score  trade_type
timestamp                                                          
2018-01-02 06:00:00+00:00  15098.88     0.144373    0.0         1.0
2018-01-03 06:00:00+00:00  15098.14    -0.000049    1.0         1.0
2018-01-04 06:00:00+00:00  15038.00    -0.003983    1.0         1.0
2018-01-05 06:00:00+00:00  16594.08     0.103477    0.0         0.0
2018-01-06 06:00:00+00:00  17098.99     0.030427    0.0         0.0
...                             ...          ...    ...         ...
2022-03-10 06:00:00+00:00  38783.79    -0.011152    1.0         1.0
2022-03-11 06:00:00+00:00  39178.11     0.010167    0.0         0.0
2022-03-12 06:00:00+00:00  39128.83    -0.001258    1.0         1.0
2022-03-13 06:00:00+00:00  38535.33    -0.015168    1.0         1.0
2022-03-14 05:00:00+00:00  38794.04     0.006714    1.0         1.0

[1533 rows x 4 columns]>

In [90]:
# Import module
from sklearn.model_selection import train_test_split

# Split training and testing sets
# Create X, or features DataFrame
features = project_df[['score']]

# Create y, or target DataFrame
target = project_df['trade_type']

# Use train_test_split to separate the data
training_features, testing_features, training_targets, testing_targets = train_test_split(features, target)

In [91]:
training_targets.head(50)

timestamp
2020-05-19 05:00:00+00:00    0.0
2019-11-06 06:00:00+00:00    0.0
2021-04-04 05:00:00+00:00    0.0
2022-02-16 06:00:00+00:00    1.0
2018-06-24 05:00:00+00:00    0.0
2021-07-24 05:00:00+00:00    0.0
2021-12-12 06:00:00+00:00    0.0
2019-01-15 06:00:00+00:00    1.0
2021-01-22 06:00:00+00:00    0.0
2018-09-12 05:00:00+00:00    0.0
2021-10-01 05:00:00+00:00    0.0
2019-03-03 06:00:00+00:00    1.0
2020-07-03 05:00:00+00:00    1.0
2019-09-04 05:00:00+00:00    1.0
2022-03-05 06:00:00+00:00    0.0
2022-02-12 06:00:00+00:00    1.0
2022-02-23 06:00:00+00:00    1.0
2021-05-26 05:00:00+00:00    1.0
2021-09-01 05:00:00+00:00    0.0
2018-01-04 06:00:00+00:00    1.0
2021-03-03 06:00:00+00:00    0.0
2020-01-30 06:00:00+00:00    0.0
2019-08-14 05:00:00+00:00    1.0
2019-05-12 05:00:00+00:00    1.0
2020-12-17 06:00:00+00:00    0.0
2018-10-20 05:00:00+00:00    0.0
2021-08-08 05:00:00+00:00    1.0
2021-06-23 05:00:00+00:00    0.0
2021-12-17 06:00:00+00:00    1.0
2020-08-25 05:00:00+00:00    1.0


In [92]:
training_features

,score
timestamp,
2020-05-19 05:00:00+00:00,0.0
2019-11-06 06:00:00+00:00,0.0
2021-04-04 05:00:00+00:00,0.0
2022-02-16 06:00:00+00:00,1.0
2018-06-24 05:00:00+00:00,0.0
...,...
2020-08-07 05:00:00+00:00,1.0
2021-04-08 05:00:00+00:00,0.0
2018-08-14 05:00:00+00:00,0.0


In [93]:
logistic_regression_model = LogisticRegression()

In [94]:
# Fit the model
logistic_regression_model.fit(training_features, training_targets)

LogisticRegression()

In [95]:
# Fit the model
logistic_regression_model.fit(training_features, training_targets)

LogisticRegression()

In [106]:
# Generate predictions from the model we just fit
predictions = logistic_regression_model.predict(training_features)

# Convert those predictions (and actual values) to a DataFrame
results_df = pd.DataFrame({"Prediction": predictions, "Actual": training_targets})
results_df.sort_index(inplace=True, ascending=False)
results_df.tail(100)

,Prediction,Actual
timestamp,,
2018-05-25 05:00:00+00:00,1.0,1.0
2018-05-24 05:00:00+00:00,0.0,0.0
2018-05-23 05:00:00+00:00,1.0,1.0
2018-05-22 05:00:00+00:00,1.0,1.0
2018-05-21 05:00:00+00:00,1.0,1.0
...,...,...
2018-01-08 06:00:00+00:00,1.0,1.0
2018-01-07 06:00:00+00:00,1.0,1.0
2018-01-04 06:00:00+00:00,1.0,1.0


In [107]:
results_df.to_csv('prediction.csv', header=True)


In [98]:
# Import the accuracy_score function
from sklearn.metrics import accuracy_score

# Calculate the model's accuracy on the test dataset
accuracy_score(training_targets, predictions)

0.999129677980853

In [99]:
plt.scatter(
x=results_df['Close']
y=results_df['Daily_Return']
c=results_df['score']
)

SyntaxError: invalid syntax (<ipython-input-99-a4e2c3fc3acc>, line 3)

In [ ]:
from sklearn.metrics import confusion_matrix

training_matrix = confusion_matrix()

In [103]:
df = pd.read_csv("prediction.csv", index_col = "timestamp", parse_dates=True, infer_datetime_format = True)

df.sort_index(inplace=True, ascending = False)

In [105]:
df[:][:"2022-03"]

,Prediction,Actual
timestamp,,
2022-03-14 05:00:00+00:00,1.0,1.0
2022-03-13 06:00:00+00:00,1.0,1.0
2022-03-12 06:00:00+00:00,1.0,1.0
2022-03-09 06:00:00+00:00,1.0,1.0
2022-03-08 06:00:00+00:00,0.0,0.0
2022-03-07 06:00:00+00:00,0.0,0.0
2022-03-06 06:00:00+00:00,1.0,1.0
2022-03-05 06:00:00+00:00,0.0,0.0
2022-03-03 06:00:00+00:00,1.0,1.0
